In [3]:
# !pip install paramiko
# !pip install scp

In [1]:
import os
import boto3
import subprocess
import numpy as np
import time
import datetime
import paramiko
import io
from scp import SCPClient, SCPException
import sys
from ast import literal_eval

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
INSTANCE_SIZE = 6

In [4]:
client = boto3.client('ec2', region_name='us-east-1')
# Create SQS client
sqs = boto3.resource('sqs')

In [45]:
def get_default_security_group(client, key_name):
    #extract key_name attribute from the security groups returned
    response = [group[key_name] for group in client.describe_security_groups()['SecurityGroups'] if group['GroupName'] == 'default']

    return response

def get_key_pairs(client, removeExisting=False):
    if removeExisting:
        client.delete_key_pair(KeyName='airscholar-key')

    keypairs = client.describe_key_pairs()['KeyPairs']
    keypair = list(filter(lambda x: x['KeyName'] == 'airscholar-key', keypairs))

    if not keypair:
        keypair = client.create_key_pair(KeyName='airscholar-key')
        f = io.StringIO(keypair['KeyMaterial'])
        data = f.read()
        file = open('labsuser.pem', 'w')
        file.write(data)
        file.close()
    else:
        keypair = keypair[0]

    return keypair

def launch_new_instance(client, keypair, count):
    response = client.run_instances(
        ImageId='ami-05723c3b9cf4bf4ff',
        InstanceType='t2.micro',
        KeyName=keypair,
        MaxCount=count,
        MinCount=count,
        Monitoring={
            'Enabled': True
        },
        SecurityGroupIds= get_default_security_group(client, key_name='GroupId')
    )
    ec2_inst_ids = [res["InstanceId"] for res in response]
    waiter = client.get_waiter('instance_running')
    waiter.wait(InstanceIds=[ec2_inst_ids])
    return ec2_inst_ids

def prepare_instances(client, keypair, count):
    ec2 = boto3.resource('ec2')
    ec2_inst_ids = []

    deployed_count = 0
    for instance in ec2.instances.all():
        deployed_count += 1
        if instance.state['Name'] == 'running':
            ec2_inst_ids.append(instance.id)

    if deployed_count < count:
        ec2_inst_ids.append(launch_new_instance(client, keypair, (count - deployed_count)))

    if not ec2_inst_ids:
        ec2_inst_ids.append(launch_new_instance(client, keypair, count))

    return ec2, ec2_inst_ids

def configure_ssh():
    sshs = []
    for count in range(0, INSTANCE_SIZE):
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        sshs.append(ssh)
    return sshs

def ssh_connect_with_retry(ssh, ip_address, retries):
    if retries > 3:
        return False
    f = open('labsuser.pem', 'r')
    privkey = paramiko.RSAKey.from_private_key(f)
    # print(privkey)
    interval = 5
    try:
        retries += 1
        print('SSH into the instance: {}'.format(ip_address))
        ssh.connect(hostname=ip_address,
                    username='ec2-user', pkey=privkey)
        return True
    except Exception as e:
        print(e)
        time.sleep(interval)
        print('Retrying SSH connection to {}'.format(ip_address))
        ssh_connect_with_retry(ssh, ip_address, retries)

def ssh_disconnect(ssh):
        """Close ssh connection."""
        if ssh:
            ssh.close()

def get_public_address(ec2, instance_id):
    # ec2 = boto3.resource('ec2', region_name='us-east-1')
    instance = ec2.Instance(id=instance_id)
    instance.wait_until_running()
    current_instance = list(ec2.instances.filter(InstanceIds=[instance_id]))
    ip_address = current_instance[0].public_ip_address
    return ip_address

def get_queue(sqs, queue_name):
    # Get the queue. This returns an SQS.Queue instance
    # There is no queue, create a new SQS queue
    attributes = {
        'DelaySeconds': '0',
        'MessageRetentionPeriod': '86400',
        "ReceiveMessageWaitTimeSeconds": "0"
    }

    for idx in range(INSTANCE_SIZE):
        sqs.create_queue(
            QueueName=f"{queue_name}{idx}",
            Attributes=attributes
        )

        sqs.create_queue(
            QueueName=f'result-queue-{idx}',
            Attributes=attributes
        )

def send_message_to_queue(sqs, queue_name, message):
    queue = sqs.get_queue_by_name(QueueName=queue_name)

    # Send message to SQS queue
    response = queue.send_messages(
        Entries=message
    )
    return response

def install_required_packages(ssh):
    stdin, stdout, stderr = ssh.exec_command("sudo yum install pip -y && sudo pip install numpy boto3 AST")
    return stdout, stderr

def get_messages_from_queue(instance_size):
    messages = []

    for idx in range(0, instance_size):
        queue_name = f'result-queue-{idx}'
        print(queue_name)
        sqs = boto3.resource('sqs')
        queue = sqs.get_queue_by_name(QueueName=queue_name)

        for message in queue.receive_messages():
            messages.append(message.body)

    return messages

def split_row(array, nrows, ncols):
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = array.shape
    assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
    assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
    return (array.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))

def split_col(array, nrows, ncols):
    """Split a matrix into sub-matrices."""
    r, h = array.shape
    return [np.vsplit(i, 5) for i in np.hsplit(arr1, r)]

def generate_array(nrows, ncols):
    arr = np.random.randint(10, size=(nrows, ncols))
    # # print('arr 1:\n', arr)
    # arr = split_row(arr, 1, split_size)
    arr1 = np.random.randint(20, size=(nrows, ncols))
    # print('arr 2:\n', arr1)
    # arr1 = split_col(arr1, 1, split_size)

    return arr, arr1

def upload_file_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except:
        # logging.error(e)
        return False
    return True

def bulk_upload(scp, filepaths: list[str], remote_path, host):
        """
        Upload multiple files to a remote directory.

        :param List[str] filepaths: List of local files to be uploaded.
        """
        try:
            scp.put(
                filepaths,
                remote_path=remote_path,
                recursive=True
            )
            print(f"Finished uploading {len(filepaths)} files to {remote_path} on {host}")
        except SCPException as e:
            print(f"SCPException during bulk upload: {e}")
        except Exception as e:
            print(f"Unexpected exception during bulk upload: {e}")

def configure_aws_access_for_ssh(ssh, ip_address):
    """
    This function extracts the AWS configuration you have locally and push to the server
    :param ssh:ssh object
    :return:
    """
    output = subprocess.getoutput("cat ~/.aws/credentials")
    ssh.exec_command(f'mkdir ~/.aws && touch ~/.aws/credentials')
    ssh.exec_command(f"echo '{output}' > ~/.aws/credentials")
    print(f'SSH AWS configuration done for {ip_address}')

def matrix_dot_product(matrix_a, matrix_b):
    start_time = datetime.datetime.now()
    result = []
    for i in range(len(matrix_a)):
        row = []
        for j in range(len(matrix_b[0])):
            sum = 0
            for k in range(len(matrix_b)):
                sum += matrix_a[i][k] * matrix_b[k][j]
            row.append(sum)
        result.append(row)
    print('Computation time', datetime.datetime.now() - start_time)

    return result

def matrix_add(matrix_1, matrix_2):
    start_time = datetime.datetime.now()
    result = []
    for idx_row in range(0, len(matrix_1)):
        row = matrix_1[idx_row]
        row1 = matrix_2[idx_row]
        cols = []
        for idx_col in range(0, len(row)):
            cols.append(row[idx_col] + row1[idx_col])
        result.append(cols)
    print('Computation time', datetime.datetime.now() - start_time)
    return result

In [6]:
QUEUE_NAME = 'queue'

In [7]:
get_queue(sqs, QUEUE_NAME)

In [46]:
sshs = configure_ssh() 
keypair = get_key_pairs(client, False)
ec2, instances = prepare_instances(client, keypair['KeyName'], INSTANCE_SIZE)
ip_addresses = [get_public_address(ec2, instance) for instance in instances]

for idx in range(0, len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    ssh_connect_with_retry(ssh, ip_address, 0)

SSH into the instance: 3.95.28.200
SSH into the instance: 107.20.122.207
SSH into the instance: 44.202.55.80
SSH into the instance: 54.172.123.86
SSH into the instance: 54.81.138.118
SSH into the instance: 54.86.228.162


In [9]:
# ssh_disconnect(ssh)
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    stdout, stderr = install_required_packages(ssh)
    # print(stdout.read().decode('utf-8'))
    # print(stderr.read().decode('utf-8'))
    configure_aws_access_for_ssh(ssh, ip_address)

SSH AWS configuration done for 3.95.28.200
SSH AWS configuration done for 107.20.122.207
SSH AWS configuration done for 44.202.55.80
SSH AWS configuration done for 54.172.123.86
SSH AWS configuration done for 54.81.138.118
SSH AWS configuration done for 54.86.228.162


In [10]:
from files.file_helper import fetch_local_files
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    scp = SCPClient(ssh.get_transport())
    bulk_upload(scp, fetch_local_files('./worker'), '~', ip_address)

Finished uploading 3 files to ~ on 3.95.28.200
Finished uploading 3 files to ~ on 107.20.122.207
Finished uploading 3 files to ~ on 44.202.55.80
Finished uploading 3 files to ~ on 54.172.123.86
Finished uploading 3 files to ~ on 54.81.138.118
Finished uploading 3 files to ~ on 54.86.228.162


## TODO: START THE WORKER ON ALL THE INSTANCES

In [11]:
stdin, stdout, stderr = sshs[0].exec_command('ls')
print(stdout.read().decode('utf-8'))

helper.py
__pycache__
queue_helper.py
worker.py



In [12]:
def start_worker(ssh, worker_id):
    stdin, stdout, stderr = ssh.exec_command(f'python ./worker.py {worker_id}')
    # result = stdout.read().decode('utf-8')
    # return result

In [29]:
stdin, stdout, stderr = sshs[0].exec_command(f'python ./worker.py 1')
print(stdout.read().decode('utf-8'))
print(stderr.read().decode('utf-8'))

Worker 1 started
Queue url: https://sqs.us-east-1.amazonaws.com/469282757936/queue1
Computation time 0:00:00.000110
Message 78 processed!

Traceback (most recent call last):
  File "/home/ec2-user/./worker.py", line 42, in <module>
    response = list([{"Id": f"{index+1}", "MessageBody": str(index, result)}])
TypeError: str() argument 2 must be str, not list



In [14]:
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    start_worker(ssh, idx)
# sqs = boto3.resource('sqs')
# sqs
# response = send_message_to_queue(sqs, QUEUE_NAME, "Hi there! This is my queue message4!", 1, 0)
# response = send_message_to_queue(sqs, QUEUE_NAME, 'X', 1, 0)

### ADDITION

In [51]:
ARRAY_SIZE = 100

In [52]:
arr, arr1 = generate_array(ARRAY_SIZE,ARRAY_SIZE)

In [55]:
INT_ARRAY_SIZE = 10 #int(ARRAY_SIZE/INSTANCE_SIZE)
s_arr = split_row(arr, INT_ARRAY_SIZE, INT_ARRAY_SIZE)
s_arr1 = split_row(arr1, INT_ARRAY_SIZE, INT_ARRAY_SIZE)

In [56]:
print(s_arr[1])
print(s_arr1[1])

[[8 7 4 4 3 0 4 5 8 6]
 [5 5 9 0 4 9 0 7 2 1]
 [0 5 3 7 0 6 1 1 5 1]
 [1 5 4 7 4 8 3 6 6 0]
 [9 4 3 4 6 6 4 1 6 0]
 [5 4 2 2 7 5 5 2 7 1]
 [4 5 8 4 8 2 9 9 1 2]
 [6 3 3 3 2 3 0 1 1 9]
 [8 3 4 2 6 6 4 6 0 6]
 [9 5 1 3 9 4 8 4 4 2]]
[[11 11  9 15  4  7 11  6 10  4]
 [14  9 15  4  1 15 18 16  6  1]
 [17 16  3 18 17  4 19 16 19  7]
 [10  9  4  8 13 10  0  0 11  1]
 [ 5  5 16  2  2  6 16  4 16 12]
 [ 9  4 16  1  7  1  7  5 11  9]
 [16 16  9  8 17  6 16 18 15  2]
 [16  2  8  0  7  7  4  7 18  3]
 [ 6 16  3 14  9 14  2 16  5 18]
 [ 1  5  2  7 16 15 15 18 16  6]]


In [20]:
# data = np.arange(0, len(s_arr))

In [21]:
# for i in np.array_split(data, 6):
#     print(i)

In [22]:
# for row in range(0, INT_ARRAY_SIZE):
#     print(s_arr[row])
#     break

In [23]:
# iter = len(s_arr)/100

In [24]:

# b = str(b).replace('\n', ',').replace(' ', '')

In [25]:
# for y in range(0, 20)
# for x in range(0, iter):
# a = arr[0]
# b = arr[1]
# a = str(a).replace(' ', ',').replace('\n', '')
# b = str(b).replace('\n', ',').replace(' ', '')
    
    # stdin, stdout, stderr = ssh.exec_command(f'python dot_prod.py {a} {b}')
    # result = stdout.read().decode("utf-8").replace('\n', '')
    # literal_eval(result)[0][0]
    # print(stderr.read().decode("utf-8"))

In [57]:
def reformat_data(data):
    return str(data).replace('\n', '')
# a = s_arr[0]
# b = s_arr[1]
# a = str(a).replace(' ', ',').replace('\n', '')
# b = str(b).replace(' ', ',').replace('\n', '')
# ROW_LENGTH = int(len(s_arr)/INSTANCE_SIZE)
# messages = []
data = np.arange(0, len(s_arr))

for id, dt in enumerate(np.array_split(data, INSTANCE_SIZE)):    
    # [print(f'queue{id}', {"Id": f"{idx+1}", "MessageBody": str((idx, (reformat_data(s_arr[idx]), reformat_data(s_arr1[idx])))) })
    #             for idx in range(min(dt), max(dt)+1)]
    [send_message_to_queue(sqs, f'queue{id}', [{"Id": f"{idx+1}", "MessageBody": str((idx, (reformat_data(s_arr[idx]), reformat_data(s_arr1[idx])))) }])
                for idx in range(min(dt), max(dt)+1)]
    
# for index in np.array_split(data, INSTANCE_SIZE):
#     # print(index)
#     messages.append([{"Id": f"{idx+1}", "MessageBody": str((idx, (reformat_data(s_arr[idx]), reformat_data(s_arr1[idx])))) }
#                 for idx in range(min(index), max(index)+1)])
    # if index <4:
    #     print(messages)
# messages1 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(5, 10)]
# messages2 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(10, 15)]
# messages3 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(15, 20)]
# messages4 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(20, 25)]

In [21]:
# messages

In [124]:
# messages[3]

In [26]:

     
# [send_message_to_queue(sqs, 'queue0', messages[idx]) for idx in range(0, 4)]
# [send_message_to_queue(sqs, 'queue1', messages[idx]) for idx in range(4, 8)]
# [send_message_to_queue(sqs, 'queue2', messages[idx]) for idx in range(8, 12)]
# [send_message_to_queue(sqs, 'queue3', messages[idx]) for idx in range(12, 16)]
# [send_message_to_queue(sqs, 'queue4', messages[idx]) for idx in range(16, 20)]
# [send_message_to_queue(sqs, 'queue5', messages[idx]) for idx in range(20, 24)]

    # print(f'queue{int((idx * INSTANCE_SIZE) % INSTANCE_SIZE)}')
        # send_message_to_queue(sqs, f'queue{int(idx/2)}', messages[idx])

In [317]:
compute_res = get_messages_from_queue(INSTANCE_SIZE)

result-queue-0
result-queue-1
result-queue-2
result-queue-3
result-queue-4
result-queue-5


In [322]:
compute_res = [literal_eval(compute) for compute in compute_res]

In [332]:
len(compute_res)

6

In [204]:
# # sqs = boto3.resource('sqs')
# queue = sqs.(QueueName='airscholar-queue')
# for message in sqs.receive_message(
#             MaxNumberOfMessages=10):
#         # process message body
#         body = json.loads(message.body)
#         print(body)


In [143]:
# message = response['Messages']
# receipt_handle = message['ReceiptHandle']

# # Delete received message from queue
# sqs.delete_message(
#     QueueUrl=queue_url,
#     ReceiptHandle=receipt_handle
# )


In [196]:
# print(json.dumps(message, indent=4))

In [45]:
# # ec2_inst_id
# bucketName = 'airscholar-mlbd-bucket'

In [146]:
# s3 = boto3.client('s3')

In [ ]:
# response = s3.get_object(Bucket=bucketName,
#                          Key='data.json')
# print("Done, response body:")
# print(response['Body'].read())

In [ ]:
#create instance
#configure instance
#create matrix
#split matrix
#send matrix to the queue
#read matrix from the queue on the instance created
#compute matrix
#send result to base


In [27]:
!git add .
!git commit -am "Update infrastructure implementation. Fixed worker computation"
!git push --set-upstream origin main

'git' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.
